In [33]:
from __future__ import print_function

import mysql.connector
from mysql.connector import Error
import pandas as pd
import os
from dotenv import load_dotenv
from datetime import date, datetime, timedelta
import sqlalchemy
from openpyxl import load_workbook

In [34]:
load_dotenv()

(host, user, password, database) = (os.getenv('host'), os.getenv('user'), os.getenv('passwd'), os.getenv('database'))

In [31]:
def create_server_connection(host_name, user_name, user_password, database):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            db = database
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

def cursor_query(cursor, query):
    with cursor as cursor:
        try:
            cursor.execute(query)
            # Fetch rows from last executed query
            result = cursor.fetchall()
            for row in result:
                print(row)
        except Error as err:
            print(f"Error: '{err}'")

def close_connection(connection):
    try:
        connection.close()
    except Error as err:
        print(f"Error: '{err}'")

In [35]:
connection = create_server_connection(host, user, password, database)
cursor = connection.cursor()
table = "ndc_product"

MySQL Database connection successful


In [30]:
show_table_query = "DESCRIBE " + table
cursor_query(cursor, show_table_query)

('key', b'int', 'NO', 'PRI', None, '')
('id', b'varchar(45)', 'NO', '', None, '')
('code', b'varchar(45)', 'NO', 'UNI', None, '')
('type', b'varchar(45)', 'NO', '', None, '')
('proprietary_name', b'varchar(45)', 'NO', '', None, '')
('proprietary_name_suffix', b'varchar(45)', 'YES', '', None, '')
('nonproprietary_name', b'varchar(45)', 'NO', '', None, '')
('dosage_form', b'varchar(45)', 'NO', '', None, '')
('route', b'varchar(45)', 'YES', '', None, '')
('start_marketing_date', b'date', 'NO', '', None, '')
('end_marketing_date', b'date', 'YES', '', None, '')
('marketing_category_name', b'varchar(45)', 'NO', '', None, '')
('application_number', b'varchar(45)', 'YES', '', None, '')
('labeler_name', b'varchar(45)', 'NO', '', None, '')
('substance_name', b'varchar(45)', 'YES', '', None, '')
('active_numenator_strength', b'varchar(45)', 'YES', '', None, '')
('active_ingredient_unit', b'varchar(45)', 'YES', '', None, '')
('pharma_class', b'varchar(45)', 'YES', '', None, '')
('descheduled', b'v

In [45]:
product_path = 'ndcxls/product.xlsx'
wb = load_workbook(product_path, use_iterators=True)
sheet = wb.worksheets[0]
row_count = sheet.max_row
start_row = 1
row_size = 1000
table = 'ndc_product'
columns = ['id', 'code', 'type', 'proprietary_name', 'proprietary_name_suffix', 'nonproprietary_name', 'dosage_form', 'route', 'start_marketing_date', 'end_marketing_date', 
    'marketing_category_name', 'application_number', 'labeler_name', 'substance_name', 'active_numenator_strength', 'active_ingredient_unit', 'pharma_class', 
    'descheduled','ndc_exclude', 'listing_cerified_through']
sqltypes = {'id': sqlalchemy.types.VARCHAR(length=64), 'code':sqlalchemy.types.VARCHAR(length=16), 'type': sqlalchemy.types.VARCHAR(length=32), 
    'proprietary_name': sqlalchemy.types.VARCHAR(length=256), 'proprietary_name_suffix': sqlalchemy.types.VARCHAR(length=128), 'nonproprietary_name': sqlalchemy.types.VARCHAR(length=512), 
    'dosage_form': sqlalchemy.types.VARCHAR(length=64), 'route': sqlalchemy.types.VARCHAR(length=128), 'start_marketing_date': sqlalchemy.DateTime() , 
    'end_marketing_date': sqlalchemy.DateTime(), 'marketing_category_name': sqlalchemy.types.VARCHAR(length=45), 'application_number': sqlalchemy.types.VARCHAR(length=20), 
    'labeler_name': sqlalchemy.types.VARCHAR(length=128), 'substance_name': sqlalchemy.types.VARCHAR(length=3450), 'active_numenator_strength': sqlalchemy.types.VARCHAR(length=510), 
    'active_ingredient_unit': sqlalchemy.types.VARCHAR(length=1500), 'pharma_class': sqlalchemy.types.VARCHAR(length=4010), 'descheduled':sqlalchemy.types.VARCHAR(length=12),
    'ndc_exclude':sqlalchemy.types.VARCHAR(length=20), 'listing_cerified_through': sqlalchemy.DateTime()}

df = pd.read_excel(product_path, header=None, names=columns, skiprows= start_row, nrows = row_size)
df.columns = columns

In [62]:
df.tail()

,id,code,type,proprietary_name,proprietary_name_suffix,nonproprietary_name,dosage_form,route,start_marketing_date,end_marketing_date,marketing_category_name,application_number,labeler_name,substance_name,active_numenator_strength,active_ingredient_unit,pharma_class,descheduled,ndc_exclude,listing_cerified_through
995,0065-4147_fabf46ca-fe61-46db-a256-9f7fc346f700,0065-4147,HUMAN PRESCRIPTION DRUG,SIMBRINZA,NaN,brinzolamide/brimonidine tartrate,SUSPENSION/ DROPS,OPHTHALMIC,2013-05-06,NaT,NDA,NDA204251,"Alcon Laboratories, Inc.",BRIMONIDINE TARTRATE; BRINZOLAMIDE,2; 10,mg/mL; mg/mL,"Adrenergic alpha-Agonists [MoA], Carbonic Anhy...",NaN,0,2023-12-31
996,0065-4274_74a99914-4b5d-4428-9184-043f0dbe4c9e,0065-4274,HUMAN OTC DRUG,PATADAY TWICE A DAY RELIEF,NaN,OLOPATADINE HYDROCHLORIDE,SOLUTION,OPHTHALMIC,2020-02-28,NaT,NDA,NDA020688,"Alcon Laboratories, Inc.",OLOPATADINE HYDROCHLORIDE,1,mg/mL,"Decreased Histamine Release [PE], Histamine H1...",NaN,0,2023-12-31
997,0065-8063_0bdcea3f-656b-44b8-82e6-f17af8f79659,0065-8063,HUMAN OTC DRUG,GenTeal Tears Moderate Preservative Free,NaN,"dextran 70, hypromellose 2910",SOLUTION/ DROPS,OPHTHALMIC,2017-12-22,NaT,OTC MONOGRAPH FINAL,part349,"Alcon Laboratories, Inc.",DEXTRAN 70; HYPROMELLOSE 2910 (4000 MPA.S),1; 3,mg/mL; mg/mL,"Increased Intravascular Volume [PE], Osmotic A...",NaN,0,2023-12-31
998,0065-8064_c964fdd3-2f59-4404-90e1-80f30e7e2743,0065-8064,HUMAN OTC DRUG,GenTeal Tears,Severe,Hypromellose,GEL,OPHTHALMIC,2019-12-14,NaT,OTC MONOGRAPH FINAL,part349,"Alcon Laboratories, Inc.",HYPROMELLOSE 2910 (4000 MPA.S),0.003,g/g,NaN,NaN,0,2023-12-31
999,0065-8067_1ad1f206-b037-4e6d-97dc-01171a64fae8,0065-8067,HUMAN OTC DRUG,GenTeal Tears Gel Drops,NaN,polyethylene glycol 400 and propylene glycol,GEL,OPHTHALMIC,2021-02-01,NaT,OTC MONOGRAPH FINAL,part349,"Alcon Laboratories, Inc.",POLYETHYLENE GLYCOL 400; PROPYLENE GLYCOL,4; 3,mg/mL; mg/mL,NaN,NaN,0,2023-12-31


In [58]:
df['start_marketing_date'] =  pd.to_datetime(df['start_marketing_date'], format='%Y%m%d')
df['end_marketing_date'] =  pd.to_datetime(df['end_marketing_date'], format='%Y%m%d')
df['listing_cerified_through'] =  pd.to_datetime(df['listing_cerified_through'], format='%Y%m%d')
df.head()

,id,code,type,proprietary_name,proprietary_name_suffix,nonproprietary_name,dosage_form,route,start_marketing_date,end_marketing_date,marketing_category_name,application_number,labeler_name,substance_name,active_numenator_strength,active_ingredient_unit,pharma_class,descheduled,ndc_exclude,listing_cerified_through
0,0002-0800_b02ed630-6947-431a-a8c8-227571403941,0002-0800,HUMAN OTC DRUG,Sterile Diluent,NaN,diluent,"INJECTION, SOLUTION",SUBCUTANEOUS,1987-07-10,NaT,BLA,BLA018781,Eli Lilly and Company,WATER,1,mL/mL,NaN,NaN,0,2023-12-31
1,0002-1200_c731b9c5-eaa5-4310-9c0a-11df9268b9ae,0002-1200,HUMAN PRESCRIPTION DRUG,Amyvid,NaN,Florbetapir F 18,"INJECTION, SOLUTION",INTRAVENOUS,2012-06-01,NaT,NDA,NDA202008,Eli Lilly and Company,FLORBETAPIR F-18,51,mCi/mL,"Positron Emitting Activity [MoA], Radioactive ...",NaN,0,2023-12-31
2,0002-1210_d03b2693-0231-4df4-a037-63017a42e85a,0002-1210,HUMAN PRESCRIPTION DRUG,TAUVID,NaN,Flortaucipir F-18,"INJECTION, SOLUTION",INTRAVENOUS,2020-05-28,NaT,NDA,NDA212123,Eli Lilly and Company,FLORTAUCIPIR F-18,51,mCi/mL,NaN,NaN,0,2023-12-31
3,0002-1220_d03b2693-0231-4df4-a037-63017a42e85a,0002-1220,HUMAN PRESCRIPTION DRUG,TAUVID,NaN,Flortaucipir F-18,"INJECTION, SOLUTION",INTRAVENOUS,2022-07-01,NaT,NDA,NDA212123,Eli Lilly and Company,FLORTAUCIPIR F-18,100,mCi/mL,NaN,NaN,0,2023-12-31
4,0002-1433_547fdc66-8f4c-422d-ac7d-b78865738ce0,0002-1433,HUMAN PRESCRIPTION DRUG,Trulicity,NaN,Dulaglutide,"INJECTION, SOLUTION",SUBCUTANEOUS,2014-09-18,NaT,BLA,BLA125469,Eli Lilly and Company,DULAGLUTIDE,0.75,mg/.5mL,"GLP-1 Receptor Agonist [EPC], Glucagon-Like Pe...",NaN,0,2023-12-31


In [59]:
df.dtypes

id                                   object
code                                 object
type                                 object
proprietary_name                     object
proprietary_name_suffix              object
nonproprietary_name                  object
dosage_form                          object
route                                object
start_marketing_date         datetime64[ns]
end_marketing_date           datetime64[ns]
marketing_category_name              object
application_number                   object
labeler_name                         object
substance_name                       object
active_numenator_strength            object
active_ingredient_unit               object
pharma_class                         object
descheduled                          object
ndc_exclude                          object
listing_cerified_through     datetime64[ns]
dtype: object

In [ ]:
table = "ndc_product"
columns = ['id', 'code', 'type', 'proprietary_name', 'proprietary_name_suffix', 'nonproprietary_name', 'dosage_form', 'route', 'start_marketing_date', 'end_marketing_date', 
    'marketing_category_name', 'application_number', 'labeler_name', 'substance_name', 'active_numenator_strength', 'active_ingredient_unit', 'pharma_class', 
    'descheduled',' ndc_exclude', 'listing_cerified_through']

insert_ndc_product = """
INSERT INTO ndc_product
    (id, code, type, proprietary_name, proprietary_name_suffix, nonproprietary_name, dosage_form, route, start_marketing_date, end_marketing_date, 
    marketing_category_name, application_number, labeler_name, substance_name, active_numenator_strength, active_ingredient_unit, pharma_class, 
    descheduled, ndc_exclude, listing_cerified_through)
VALUES ( %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

In [37]:
close_connection(connection)